# Problem Set 4

This problem set touches the problem of assessing the income process. 

### Question 1. Income Process Moment Equation

**(a)** 
\begin{align}
g_{it} &= \Delta u_{it} \\
&= p_{it}+m_{it}-p_{it-1}-m_{it-1}\\
&=m_{it}-m_{it-1}+\zeta_{it}
\end{align}

Similarly, 
\begin{align}
g_{it-1} = m_{it-1}-m_{it-2}+\zeta_{it-1}
\end{align}

Therefore, 
\begin{align}
E(g_{it}g_{it}) = 2\sigma_m^2 + \sigma_{\zeta}^2
\end{align}

And,
\begin{align}
E(g_{it}g_{it-1}) = -\sigma_m^2
\end{align}

Combining the two moment conditions above, the following can be derived:
\begin{align}
\sigma_\zeta^2 = E(g_{it}g_{it}) + 2E(g_{it}g_{it-1})
\end{align}

And 
\begin{align}
\sigma_m^2 = -E(g_{it}g_{it-1})
\end{align}

**(b)** 
We need to derived the following two equations:
\begin{align}
E(g_{it}g_{it}|L_{it}=1,L_{it-1}=1) &= 2\sigma_m^2 + E(\zeta_{it}^2|L_{it} =1,L_{it-1}=1)\\
E(g_{it}g_{it-1}|L_{it}=1,L_{it-1}=1) &= -\sigma_m^2 + E(\zeta_{it}\zeta_{it-1}|L_{it}=1,L_{it-1}=1) \\
\end{align}

We first consider the first equation:
\begin{align}
E(g_{it}^2|\eta_{it}>-x_{it}'\gamma,\eta_{it-1}>-x_{it-1}'\gamma) &= 2\sigma_m^2 + E(\zeta_{it}^2|\eta_{it}>-x_{it}'\gamma,\eta_{it-1}>-x_{it-1}'\gamma)
\end{align}


We need to compute $E(\zeta_{it}^2|\eta_{it}>-x_{it}'\gamma,\eta_{it-1}>-x_{it-1}'\gamma)$ as follows:
\begin{align}
E(\zeta_{it}^2|\eta_{it}>-x_{it}'\gamma,\eta_{it-1}>-x_{it-1}'\gamma) &= E((cov(\zeta_{it},\eta_{it})\eta_{it})^2+ z^2 + 2\rho\sigma_{\zeta}\eta z|\eta_{it}>-x_{it}'\gamma,\eta_{it-1}>-x_{it-1}'\gamma)
\end{align}
where z is such that $\zeta_{it} = cov(\zeta_{it},\eta_{it})\eta_{it}+z$ and $\sigma_z^2 = \sigma_{\zeta}^2(1-\rho^2)$

Therefore, 
\begin{align}
E(\zeta_{it}^2|\eta_{it}>-x_{it}'\gamma) = \rho^2\sigma_{\zeta}^2\left(c\dfrac{\phi(c)}{1-\Phi(c)}+1\right)+\sigma_{\zeta}^2(1-\rho^2)
\end{align}
where $c=-x_{it}'\gamma$

And thus **the first moment condition** is:
\begin{align}
E(g_{it}^2|\eta_{it}>-x_{it}'\gamma,\eta_{it-1}>-x_{it-1}'\gamma) &= 2\sigma_{m}^2 + \rho^2\sigma_{\zeta}^2\left(c\dfrac{\phi(c)}{1-\Phi(c)}+1\right)+\sigma_{\zeta}^2(1-\rho^2)\\
&= 2\sigma_{m}^2 + \sigma_{\zeta\eta}^2(-x_{it}'\gamma)\lambda_{it}+\sigma_{\zeta}^2
\end{align}
where $\lambda_{it} = \dfrac{\phi(-x_{it}'\gamma)}{1-\Phi(-x_{it}'\gamma)}$

And **the second moment condition** is:
\begin{align}
E(g_{it}g_{it-1}|\eta_{it}>-x_{it}'\gamma,\eta_{it-1}>-x_{it-1}'\gamma) = -\sigma_m^2 + \sigma_{\zeta\eta}^2(\lambda_{it}\lambda_{it-1})
\end{align}

In addition, **the third moment condition** is about the observables and the model:
\begin{align}
E(\Delta \log w_{it}|L_{it}=1,L_{it-1}=1) = \Delta z_{it}'\beta + \sigma_{\zeta\eta}\dfrac{\phi(-x_{it}'\gamma)}{1-\Phi(-x_{it}'\gamma)}
\end{align}

### Question 2. Estimate Income Process Using Nonlinear Least Squares

First, import all tools needed. 

In [182]:
import pandas as pd
import os
import numpy as np
import pandas as pd
from scipy.interpolate import interp1d, interp2d
from scipy.optimize import minimize_scalar,minimize
import matplotlib.pyplot as plt
%matplotlib inline
from quantecon.markov import DiscreteDP
from scipy.stats import norm
from math import sqrt
import matplotlib.pyplot as plt
from scipy import optimize
from scipy.optimize import brentq
import time
import statsmodels.api as sm

In [183]:
df = pd.read_csv('Pset4Data.csv')

In [184]:
df['educ'] = df['educ'].astype('category')
df = pd.get_dummies(df,drop_first = True)
df.rename(columns={'educ_High School Graduate':'HighSchool','educ_Postgraduate':'PostGraduate','educ_Some College':'SomeCollege'},inplace=True)
df.head()

,Unnamed: 0,id,wave,age,noveliv,lninc,work,HighSchool,PostGraduate,SomeCollege
0,1,1,1,31,-0.634240,-5.687436,True,0,0,0
1,2,2,1,23,1.076020,-1.690294,True,0,0,0
2,3,3,1,33,0.918417,-8.197371,True,0,1,0
3,4,4,1,29,0.303140,-5.464115,True,0,0,0
4,5,5,1,49,0.697175,-28.852108,True,0,1,0


In [185]:
df['age2'] = df['age']**2 
Labor = df.work
π = df[['age','age2','HighSchool','PostGraduate','SomeCollege','noveliv']]
π = sm.add_constant(π)

C:\Users\zhuang13\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:2389: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


In [186]:
probit_model = statsmodels.discrete.discrete_model.Probit(Labor,π.astype(float)).fit()
probit_coefficient = probit_model.params
df['FittedValue'] = probit_model.fittedvalues

Optimization terminated successfully.
         Current function value: 0.120995
         Iterations 13


In [187]:
probit_coefficient

const          -0.414992
age             0.534506
age2           -0.011089
HighSchool     -0.321881
PostGraduate    0.148578
SomeCollege    -0.177513
noveliv         0.811032
dtype: float64

In [188]:
lambda_i = norm.pdf(-df['FittedValue'])/(1-norm.cdf(-df['FittedValue']))

In [189]:
df['lambda'] = lambda_i

In [190]:
df['lambda_lag'] = df.groupby('id')['lambda'].shift()
df['d_income'] = df['lninc'] - df.groupby('id')['lninc'].shift()
df['d_age'] = df['age'] - df.groupby('id')['age'].shift()
df['d_age2'] = df['age2'] - df.groupby('id')['age2'].shift()

In [191]:
Y = df.d_income
X = df[['d_age2','lambda']]
X = sm.add_constant(X)

In [192]:
OLS_model = sm.OLS(Y.astype(float),X.astype(float),missing='drop')
results = OLS_model.fit()
results.params

const     0.403554
d_age2   -0.020076
lambda    0.031588
dtype: float64

In [193]:
Δu = df['d_income'] - results.params.const - df['d_age2']*results.params.d_age2

In [194]:
df['Δu'] = Δu
df['lag_Δu'] = df.groupby('id')['Δu'].shift()
df['Δu_2'] = df['Δu']**2

In [195]:
cov = results.params['lambda']
cov

0.03158805983619616

In [196]:
E_inter = np.mean(df['Δu']*df['lag_Δu'])
σ_m2 = cov**2*np.mean(df['lambda']*df['lambda_lag']) - E_inter
σ_m2

0.020966598281527228

In [197]:
E_Δu2 = np.mean(df['Δu']**2)
σ_ζ2 = E_Δu2 - 2*σ_m2 - cov**2*np.mean(-df['FittedValue']*df['lambda'])
σ_ζ2

0.02629186087496656

In [ ]:
# bootstrap
N = 3000
nb = 100
b_E_inter = np.zeros((nb))
b_σ_m2 = np.zeros((nb))
b_E_Δu2 = np.zeros((nb))
b_ζ2 = np.zeros((nb))
for k in range(nb):
    draw = np.random.choice(N, N, replace = True)
    df_draw = df[df.id==draw[0]]
    for i in range(1,N):
        df_draw = df_draw.append(df[df.id==draw[i]])
        
    b_E_inter[k] = np.mean(df_draw['Δu']*df_draw['Δu'])
    b_σ_m2[k] = cov**2*np.mean(dr_draw['lambda']*df_draw['lambda']) - b_E_inter[k]
    b_E_Δu2[k] = np.mean(df_draw['Δu']**2)
    b_ζ2[k] = b_E_Δu2[k] - 2*b_σ_m2[k] - cov**2*np.mean(-df_draw['FittedValue']*df_draw['lambda'])